In [96]:
import pandas as pd

def calculate_total_payout_naive(df):
    # Initialize variables
    total_payout = 0
    num_hit = 0
    num_miss = 0

    # Prepare a list to collect rows for results
    filtered_rows = []

    # Iterate over rows
    for _, row in df.iterrows():
        if (
            row["outcome"] == row["contract_type"]
            and row["pdf_estimate"] >= row["current_kalshi_price"] + 10
            and row["current_kalshi_price"] >= 50
        ):
            total_payout += 100 / row["current_kalshi_price"] - 1
            num_hit += 1
            filtered_rows.append(row)  # Collect the row
        elif (
            row["outcome"] != row["contract_type"]
            and row["pdf_estimate"] >= row["current_kalshi_price"] + 10
            and row["current_kalshi_price"] >= 50
        ):
            total_payout -= 1
            num_miss += 1
            filtered_rows.append(row)  # Collect the row

    # Convert the collected rows into a DataFrame
    res = pd.DataFrame(filtered_rows)

    # Print the total payout
    print(f"Net Profit: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")

    # Return results for further processing
    return total_payout, num_hit, num_hit + num_miss, res


In [97]:
df = pd.read_csv("kalshi-historical-3-30-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])
df =  df[df['time'] > pd.Timestamp('2025-03-23')] 
df

,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
77320,2025-03-24 11:27:28,BTC,Daily,No,89250,70.92,90,No
77321,2025-03-24 11:27:28,BTC,Daily,Yes,89250,29.08,16,No
77322,2025-03-24 11:27:28,BTC,Daily,No,88750,58.15,79,No
77323,2025-03-24 11:27:28,BTC,Daily,Yes,88750,41.85,28,No
77324,2025-03-24 11:27:28,BTC,Daily,No,88250,45.98,62,No
...,...,...,...,...,...,...,...,...
117127,2025-03-30 15:30:08,BTC,Daily,Yes,83250,46.47,18,No
117128,2025-03-30 15:30:08,BTC,Daily,No,82750,41.18,66,No
117129,2025-03-30 15:30:08,BTC,Daily,Yes,82750,58.82,42,No
117130,2025-03-30 15:30:08,BTC,Daily,No,82250,29.93,34,Yes


In [98]:
_, _, _, res_df = calculate_total_payout_naive(df)
res_df

Net Profit: 792.5048570216084
Total Trades: 3852/4608


,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
77825,2025-03-24 13:18:04,BTC,Daily,Yes,88250,61.31,50,No
77833,2025-03-24 13:20:05,BTC,Daily,Yes,88250,61.00,51,No
77925,2025-03-24 13:44:05,BTC,Daily,Yes,88250,64.77,54,No
77931,2025-03-24 13:46:05,BTC,Daily,Yes,88250,62.42,50,No
78061,2025-03-24 14:24:05,BTC,Daily,Yes,88250,64.16,51,No
...,...,...,...,...,...,...,...,...
116559,2025-03-30 13:02:04,BTC,Daily,Yes,82250,65.77,53,Yes
116569,2025-03-30 13:04:04,BTC,Daily,Yes,82250,65.37,53,Yes
116619,2025-03-30 13:14:05,BTC,Daily,Yes,82250,66.44,56,Yes
116757,2025-03-30 13:46:04,BTC,Daily,Yes,82250,67.01,55,Yes


In [15]:
# 
def calculate_total_payout_three_daily_limit_with_no_same_position(df):
    # Initialize variables
    total_payout = 0
    num_hit = 0
    num_miss = 0

    # Prepare a list to collect rows for results
    filtered_rows = []

    # Track active trades per day
    trades_per_day = {}

    # Iterate over rows
    for _, row in df.iterrows():
        # Extract date from time (assuming 'time' is in datetime format)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"])

        # Check if we already have 3 active trades for this day
        if contract_key not in trades_per_day.get(trade_day, []):
            # Entry condition: Model prediction >= current kalshi price + 10 and kalshi price >= 50
            if (
                row["pdf_estimate"] >= row["current_kalshi_price"] + 10
                and row["current_kalshi_price"] >= 50
            ):
                # Register the trade for that day
                if trade_day not in trades_per_day:
                    trades_per_day[trade_day] = []
                if len(trades_per_day[trade_day]) < 3:  # Only allow 3 trades per day
                    trades_per_day[trade_day].append(contract_key)
                    
                    # Calculate payout based on outcome
                    if row["outcome"] == row["contract_type"]:
                        total_payout += 100 / row["current_kalshi_price"] - 1  # Winning payout
                        num_hit += 1
                    elif row["outcome"] != row["contract_type"]:
                        total_payout -= 1  # Losing payout
                        num_miss += 1

                    # Collect the row for results
                    filtered_rows.append(row)  # Add this trade row

    # Convert the collected rows into a DataFrame for results
    res = pd.DataFrame(filtered_rows)

    # Print the total payout and results summary
    print(f"Net Profit: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")

    # Return results for further processing
    return total_payout, num_hit, num_hit + num_miss, res


In [16]:
_, _, _, res_df = calculate_total_payout_three_daily_limit_with_no_same_position(df)
res_df

Net Profit: -9.94210205796424
Total Trades: 24/53
Winning Trades: 24
Losing Trades: 29


,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
271,2025-01-16 09:44:00,BTC,Daily,Yes,98250,65.01,52,Yes
273,2025-01-16 09:44:00,BTC,Daily,Yes,97750,80.68,64,Yes
291,2025-01-16 09:46:00,BTC,Daily,Yes,97250,78.65,66,Yes
3625,2025-01-20 13:20:00,BTC,Daily,Yes,103750,64.00,51,No
7063,2025-01-21 11:10:00,BTC,Daily,No,105000,64.42,52,Yes
7085,2025-01-21 11:12:00,BTC,Daily,No,106000,70.93,57,Yes
9742,2025-01-23 09:34:00,BTC,Daily,No,103750,67.26,55,No
10179,2025-01-23 10:10:00,BTC,Daily,Yes,103250,74.69,53,No
10181,2025-01-23 10:10:00,BTC,Daily,Yes,102750,78.73,61,Yes
13470,2025-01-24 14:46:00,BTC,Daily,Yes,105750,66.51,52,No


In [17]:
def calculate_total_payout_three_daily_limit_with_limit_sells(df):
    # Initialize variables
    total_payout = 0
    num_hit = 0
    num_miss = 0

    # Prepare a list to collect rows for results
    filtered_rows = []

    # Track active trades per day
    trades_per_day = {}

    # Store active trades with their limit order prices
    active_trades = {}

    # Iterate over rows
    for _, row in df.iterrows():
        # Extract date from time (assuming 'time' is in datetime format)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"])

        # Check if we already have 3 active trades for this day
        if contract_key not in trades_per_day.get(trade_day, []):
            # Entry condition: Model prediction >= current kalshi price + 10 and kalshi price >= 50
            if (
                row["pdf_estimate"] >= row["current_kalshi_price"] + 10
                and row["current_kalshi_price"] >= 50
            ):
                # Register the trade for that day
                if trade_day not in trades_per_day:
                    trades_per_day[trade_day] = []
                if len(trades_per_day[trade_day]) < 3:  # Only allow 3 trades per day
                    trades_per_day[trade_day].append(contract_key)

                    # Set a limit order to exit at pdf_estimate - 5
                    limit_price = row["pdf_estimate"] - 5

                    # Record the trade in active_trades
                    active_trades[contract_key] = {
                        "entry_time": row["time"],
                        "entry_price": row["current_kalshi_price"],
                        "limit_price": limit_price,
                        "entered": True,
                        "exit_time": None,
                        "exit_price": None,
                        "outcome": None,
                    }

                    # Collect the row for results
                    filtered_rows.append(row)

        # Check for limit order exit condition in active trades
        if contract_key in active_trades and active_trades[contract_key]["entered"]:
            # If the price exceeds the limit, exit the trade
            if row["current_kalshi_price"] >= active_trades[contract_key]["limit_price"]:
                # Record the exit details
                active_trades[contract_key]["exit_time"] = row["time"]
                active_trades[contract_key]["exit_price"] = row["current_kalshi_price"]

                # Calculate gain or loss
                if row["outcome"] == row["contract_type"]:
                    total_payout += 100 / row["current_kalshi_price"] - 1  # Winning payout
                    num_hit += 1
                elif row["outcome"] != row["contract_type"]:
                    total_payout -= 1  # Losing payout
                    num_miss += 1

                # Mark the trade as exited
                active_trades[contract_key]["entered"] = False

                # Add the row to the filtered results
                filtered_rows.append(row)

    # Convert the collected rows into a DataFrame for results
    res = pd.DataFrame(filtered_rows)

    # Print the total payout and results summary
    print(f"Net Profit: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")

    # Return results for further processing
    return total_payout, num_hit, num_hit + num_miss, res


In [18]:
df = pd.read_csv("kalshi-historical-3-16-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])

_, _, _, res_df = calculate_total_payout_three_daily_limit_with_limit_sells(df)
res_df

Net Profit: -5.567916636941719
Total Trades: 28/46
Winning Trades: 28
Losing Trades: 18


,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
271,2025-01-16 09:44:00,BTC,Daily,Yes,98250,65.01,52,Yes
273,2025-01-16 09:44:00,BTC,Daily,Yes,97750,80.68,64,Yes
291,2025-01-16 09:46:00,BTC,Daily,Yes,97250,78.65,66,Yes
589,2025-01-16 10:22:00,BTC,Daily,Yes,97250,66.85,75,Yes
799,2025-01-16 10:52:00,BTC,Daily,Yes,98250,56.21,64,Yes
...,...,...,...,...,...,...,...,...
69119,2025-03-13 14:24:05,BTC,Daily,Yes,80500,66.44,67,No
69919,2025-03-14 10:00:04,BTC,Daily,Yes,83250,62.56,50,Yes
69967,2025-03-14 10:06:05,BTC,Daily,Yes,83250,54.47,58,Yes
71523,2025-03-14 15:24:05,BTC,Daily,Yes,84750,63.66,50,No


In [108]:
import pandas as pd

def calculate_total_payout_limits(df, unit_size=1):
    total_payout = 0
    num_hit = 0
    num_miss = 0

    filtered_rows = []

    # Track active trades for each day (max 3 trades per day)
    trades_per_day = {}

    # Store active trades (contract_key: (entry_time, entry_price))
    active_trades = {}

    # Iterate over the rows in the dataframe
    for idx, row in df.iterrows():
        # Get the current day (we only care about the date part)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"], trade_day)
        
        # Ensure we have a maximum of 3 trades per day
        if trade_day not in trades_per_day:
            trades_per_day[trade_day] = []

        # Entry condition: If we haven't reached 3 trades for the day and the entry condition is met
        if True:
            if row["pdf_estimate"] >= row["current_kalshi_price"] + 10 and row["current_kalshi_price"] >= 50:
                # Register the trade for the day
                trades_per_day[trade_day].append(contract_key)

                # Store the trade entry details
                active_trades[contract_key] = {
                    "entry_time": row["time"],
                    "entry_price": row["current_kalshi_price"],
                    "limit_price": row["pdf_estimate"],
                    "exited": False,
                    "exit_time": None,
                    "exit_price": None,
                    "outcome": row["outcome"]
                }
                # Add this row to the filtered rows (this is the trade entry)
                filtered_rows.append(row)

        # Now we need to check if a limit sell opportunity exists for each active trade
        for trade_contract_key, trade in list(active_trades.items()):
            # Look ahead for a limit sell opportunity (price >= entry_price - 5)
            if not trade["exited"]:
                if trade_day == trade["entry_time"].date() and row["time"] > trade["entry_time"] and trade_contract_key == contract_key and row["current_kalshi_price"] >= trade["limit_price"]:
                    # Mark the trade as exited
                    trade["exit_time"] = row["time"]
                    trade["exit_price"] = trade["limit_price"]
                    trade["exited"] = True

                    # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                    # print(row["time"],row["current_kalshi_price"], contract_key)


                    # Calculate profit (gain or loss)
                    gain = (trade["exit_price"] / trade["entry_price"]) - 1
                    total_payout += gain * unit_size
                    num_hit += 1

    # Handle any remaining active trades at the end of the last day in the dataset
    for trade_contract_key, trade in list(active_trades.items()):
        if not trade["exited"]:
            if trade_contract_key[0] != trade["outcome"]:
                total_payout -= 1  # Apply -1 penalty for missed exit
                num_miss += 1
                trade["exited"] = True
                print(trade_contract_key, trade)
            else:
                total_payout += (trade["limit_price"] / trade["entry_price"]) - 1
                num_hit += 1
                trade["exited"] = True

    # Convert the filtered rows into a DataFrame for results
    res_df = pd.DataFrame(filtered_rows)

    # Print the final summary
    print(f"Net Profit in Units: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")

    return total_payout, num_hit, num_hit + num_miss, res_df



In [109]:
df = pd.read_csv("kalshi-historical-3-30-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])
_, _, _, res_df = calculate_total_payout_limits(df)
res_df

('No', 105000, datetime.date(2025, 1, 21)) {'entry_time': Timestamp('2025-01-21 11:42:00'), 'entry_price': 52, 'limit_price': 62.39, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'Yes'}
('Yes', 103750, datetime.date(2025, 1, 23)) {'entry_time': Timestamp('2025-01-23 15:52:00'), 'entry_price': 52, 'limit_price': 64.07, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 105750, datetime.date(2025, 1, 24)) {'entry_time': Timestamp('2025-01-24 14:46:00'), 'entry_price': 52, 'limit_price': 66.50999999999999, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 100750, datetime.date(2025, 1, 28)) {'entry_time': Timestamp('2025-01-28 16:46:00'), 'entry_price': 59, 'limit_price': 73.57, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 105500, datetime.date(2025, 1, 30)) {'entry_time': Timestamp('2025-01-30 15:36:00'), 'entry_price': 55, 'limit_price': 71.27, 'exited': True, 'exit_time': N

,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
271,2025-01-16 09:44:00,BTC,Daily,Yes,98250,65.01,52,Yes
273,2025-01-16 09:44:00,BTC,Daily,Yes,97750,80.68,64,Yes
289,2025-01-16 09:46:00,BTC,Daily,Yes,97750,78.65,54,Yes
291,2025-01-16 09:46:00,BTC,Daily,Yes,97250,78.65,66,Yes
293,2025-01-16 09:46:00,BTC,Daily,Yes,96750,89.21,76,Yes
...,...,...,...,...,...,...,...,...
116559,2025-03-30 13:02:04,BTC,Daily,Yes,82250,65.77,53,Yes
116569,2025-03-30 13:04:04,BTC,Daily,Yes,82250,65.37,53,Yes
116619,2025-03-30 13:14:05,BTC,Daily,Yes,82250,66.44,56,Yes
116757,2025-03-30 13:46:04,BTC,Daily,Yes,82250,67.01,55,Yes


In [105]:
import pandas as pd
import numpy as np
from datetime import time
from collections import defaultdict


def calculate_total_payout_limits_and_price_history(df, unit_size=1):
    total_payout = 0
    num_hit = 0
    num_miss = 0

    filtered_rows = []

    # Track active trades for each day (max 3 trades per day)
    trades_per_day = {}

    # Store active trades (contract_key: (entry_time, entry_price))
    active_trades = defaultdict(list)

    price_history = {}
    stop_loss_cutoff = time(15, 30)  # "15:30" -> time(15, 30)


    # Iterate over the rows in the dataframe
    for idx, row in df.iterrows():
        # Get the current day (we only care about the date part)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"], trade_day)
        # Initialize price history for the asset if not already tracked
        if contract_key not in price_history:
            price_history[contract_key] = []

        # Add current price to the price history for the asset
        price_history[contract_key].append(row["current_kalshi_price"])

        # Calculate the volatility based on the last 10 prices (adjust window size if needed)
        if len(price_history[contract_key]) > 10:
            volatility = np.std(price_history[contract_key][-10:])
        else:
            volatility = 0  # Not enough data for volatility calculation
            
        # Ensure we have a maximum of 3 trades per day
        if trade_day not in trades_per_day:
            trades_per_day[trade_day] = []

        # Entry condition: If we haven't reached 3 trades for the day and the entry condition is met
        if len(trades_per_day[trade_day]) < 99 and row["time"].time() <= stop_loss_cutoff:
            if row["pdf_estimate"] >= row["current_kalshi_price"] + 10 and row["current_kalshi_price"] >= 50:
                # Register the trade for the day
                trades_per_day[trade_day].append(contract_key)

                # Store the trade entry details
                active_trades[contract_key].append({
                    "entry_time": row["time"],
                    "entry_price": row["current_kalshi_price"],
                    "limit_price": row["pdf_estimate"],
                    "exited": False,
                    "exit_time": None,
                    "exit_price": None,
                    "outcome": row["outcome"],
                    "trailing_stop": row["current_kalshi_price"] - volatility * 3
                })
                # Add this row to the filtered rows (this is the trade entry)
                filtered_rows.append(row)

        # Now we need to check if a limit sell opportunity exists for each active trade
        for trade_contract_key, trades in list(active_trades.items()):
            for trade in trades:
                # Look ahead for a limit sell opportunity (price >= entry_price - 5)
                if not trade["exited"]:
                    if trade_day == trade["entry_time"].date() and row["time"] > trade["entry_time"] and trade_contract_key == contract_key:
                        if row["current_kalshi_price"] >= trade["limit_price"]:

                            # Mark the trade as exited
                            trade["exit_time"] = row["time"]
                            trade["exit_price"] = trade["limit_price"]
                            trade["exited"] = True

                            # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                            # print(row["time"],row["current_kalshi_price"], contract_key)

                            # Calculate profit (gain or loss)
                            gain = (trade["exit_price"] / trade["entry_price"]) - 1
                            total_payout += gain * unit_size
                            num_hit += 1
                        if row["current_kalshi_price"] <= trade["trailing_stop"] and row["time"].time() >= stop_loss_cutoff:
                            # Mark the trade as exited
                            trade["exit_time"] = row["time"]
                            trade["exit_price"] = row["current_kalshi_price"]
                            trade["exited"] = True

                            # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                            # print(row["time"],row["current_kalshi_price"], contract_key)

                            # Calculate profit (gain or loss)
                            gain = (trade["entry_price"] - row["current_kalshi_price"])/trade["entry_price"]
                            # print(trade_contract_key, trade)
                            total_payout -= gain * unit_size
                            num_miss += 1

    # Handle any remaining active trades at the end of the last day in the dataset
    for trade_contract_key, trades in list(active_trades.items()):
        for trade in trades:
            if not trade["exited"]:
                if trade_contract_key[0] != trade["outcome"]:
                    total_payout -= 1  # Apply -1 penalty for missed exit
                    num_miss += 1
                    trade["exited"] = True
                    print(trade_contract_key, trade)
                else:
                    total_payout += (trade["limit_price"] / trade["entry_price"]) - 1
                    num_hit += 1
                    trade["exited"] = True

    # Convert the filtered rows into a DataFrame for results
    res_df = pd.DataFrame(filtered_rows)

    # Print the final summary
    print(f"Net Profit in Units: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")
    return total_payout, num_hit, num_hit + num_miss, res_df



In [106]:
df = pd.read_csv("kalshi-historical-3-30-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])
cutoff_date = "2025-03-23"
df = df[df["time"] > cutoff_date]
_, _, _, res_df = calculate_total_payout_limits_and_price_history(df)
res_df

('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:38:04'), 'entry_price': 53, 'limit_price': 67.38, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(42.955598574330075)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:40:04'), 'entry_price': 52, 'limit_price': 67.44, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(42.0501256289338)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:42:06'), 'entry_price': 51, 'limit_price': 65.33, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(41.782625102557674)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:44:05'), 'entry_price': 51, 'limit_price': 63.57, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(43.6454

,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
77825,2025-03-24 13:18:04,BTC,Daily,Yes,88250,61.31,50,No
77833,2025-03-24 13:20:05,BTC,Daily,Yes,88250,61.00,51,No
77925,2025-03-24 13:44:05,BTC,Daily,Yes,88250,64.77,54,No
77931,2025-03-24 13:46:05,BTC,Daily,Yes,88250,62.42,50,No
78061,2025-03-24 14:24:05,BTC,Daily,Yes,88250,64.16,51,No
...,...,...,...,...,...,...,...,...
116559,2025-03-30 13:02:04,BTC,Daily,Yes,82250,65.77,53,Yes
116569,2025-03-30 13:04:04,BTC,Daily,Yes,82250,65.37,53,Yes
116619,2025-03-30 13:14:05,BTC,Daily,Yes,82250,66.44,56,Yes
116757,2025-03-30 13:46:04,BTC,Daily,Yes,82250,67.01,55,Yes


In [107]:
daily_counts = res_df.groupby(res_df["time"].dt.date).size()
daily_counts

time
2025-03-24     953
2025-03-25     542
2025-03-26     537
2025-03-27    1275
2025-03-28     160
2025-03-29    1026
2025-03-30     115
dtype: int64

In [100]:
# dates_to_drop = ['2025-02-04', '2025-02-05']

# # Filter out rows with the specified dates
# df = df[~df['time'].dt.date.isin(pd.to_datetime(dates_to_drop).date)]

In [ ]:
# df.to_csv('kalshi-historical-2-4-25-with-y-REMOVED-STALE_DAYS.csv', index=False)